In [27]:
from parsing import parser
from parsing_ast import transformer
import parsing_ast

In [132]:
input_str = """ 
a = 10;
"""

In [133]:
tree = parser.parse(input_str)
ast = transformer.transform(tree)

DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:literal - [Token('INTEGER_CONSTANT', '10')]
DEBUG:parsing_ast:expression - [literal('10')]
DEBUG:parsing_ast:expression_statement - [assignment(identifier('a'), literal('10')), Token('END_OF_STATEMENT', ';')]
DEBUG:parsing_ast:Processing 'statement' with [assignment(identifier('a'), literal('10'))]
DEBUG:parsing_ast:Processing 'statements' with [assignment(identifier('a'), literal('10'))]


['a', '=', '10', ';']


In [134]:
ast

statements([assignment(identifier('a'), literal('10'))])

In [135]:
class Symbol(object):
    def __init__(self, name, type=None, category=None) -> None:
        self.name = name
        self.type = type
        self.category = category

In [136]:
class BuiltinTypeSymbol(Symbol):
    def __init__(self, name) -> None:
        super().__init__(name)

    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return "<{class_name}(name='{name}')>".format(
            class_name=self.__class__.__name__,
            name=self.name,
        )

In [137]:
class VarSymbol(Symbol):
    def __init__(self, name, type=None, category=None) -> None:
        super().__init__(name, type, category)
    
    def __str__(self) -> str:
        return "<{class_name}(name='{name}', type='{type}', category='{category}')>".format(
            class_name=self.__class__.__name__,
            name=self.name,
            type=self.type,
            category=self.category
        )
    
    __repr__ = __str__

In [138]:
class SymbolTable(object):
    def __init__(self) -> None:
        self._symbols = {}
        self._init_builtins()

    def _init_builtins(self) -> None:
        self.insert(BuiltinTypeSymbol('INTEGER'))
        self.insert(BuiltinTypeSymbol('REAL'))
        self.insert(BuiltinTypeSymbol('STRING'))
        self.insert(BuiltinTypeSymbol('BOOLEAN'))

    def __str__(self) -> str:
        symtab_header = 'Symbol table contents'
        lines = ['\n', symtab_header, '_' * len(symtab_header)]
        lines.extend(
            ('%7s: %r' % (key, value))
            for key, value in self._symbols.items()
        )
        lines.append('\n')
        s = '\n'.join(lines)
        return s
    
    __repr__ = __str__

    def insert(self, symbol) -> None:
        print('Insert: %s' % symbol.name)
        self._symbols[symbol.name] = symbol

    def lookup(self, name) -> Symbol:
        print('Lookup: %s' % name)
        return self._symbols.get(name, None)

In [139]:
symbol_table = SymbolTable()

Insert: INTEGER
Insert: REAL
Insert: STRING
Insert: BOOLEAN


In [142]:
def traverse_ast(node, indent="", last=True):
    if isinstance(node, list):
        for idx, item in enumerate(node):
            if idx == len(node) - 1:
                traverse_ast(item, indent, last=True)
            else:
                traverse_ast(item, indent, last=False)
        return
    print(type(node))
    if isinstance(node, parsing_ast.VariableDeclaration):
        for name in node.identifier:
            if symbol_table.lookup(name.name):
                raise Exception('Error: Duplicate identifier found: %s' % name.name)
            symbol_table.insert(VarSymbol(name.name, node.declaration_type.value))
    if isinstance(node, parsing_ast.Declaration):
        if symbol_table.lookup(node.identifier.name):
            raise Exception('Error: Duplicate identifier found: %s' % node.identifier.name)
        symbol_table.insert(VarSymbol(node.identifier.name, node.declaration_type.value))
    if isinstance(node, parsing_ast.Assignment):
        if not symbol_table.lookup(node.identifier.name):
            raise Exception('Error: Symbol not found: %s' % node.identifier.name)
    if not hasattr(node, '__dict__'):
        return 
    children = [(k, v) for k, v in node.__dict__.items() if not k.startswith('_') and v]
    for idx, (attr, child) in enumerate(children):
        is_last = idx == len(children) - 1
        traverse_ast(child, last=is_last)

In [143]:
traverse_ast(ast)

<class 'parsing_ast.Statements'>
<class 'parsing_ast.Assignment'>
Lookup: a


Exception: Error: Symbol not found: a

In [109]:
symbol = symbol_table.lookup('i')
symbol

Lookup: i


<VarSymbol(name='i', type='var', category='None')>

In [144]:
symbol_table._symbols

{'INTEGER': <BuiltinTypeSymbol(name='INTEGER')>,
 'REAL': <BuiltinTypeSymbol(name='REAL')>,
 'STRING': <BuiltinTypeSymbol(name='STRING')>,
 'BOOLEAN': <BuiltinTypeSymbol(name='BOOLEAN')>}